# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lospalos,-8.5186,127.0025,20.70,79,41,3.40,TL,1694959906
1,1,hithadhoo,-0.6000,73.0833,28.16,78,47,7.19,MV,1694959907
2,2,abepura,-2.6333,140.5833,25.45,83,40,1.03,ID,1694959907
3,3,gamay,12.3854,125.2979,26.59,85,41,1.28,PH,1694959907
4,4,novyy urgal,51.0736,132.5642,12.19,74,100,1.10,RU,1694959907


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna(how="any")

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,lihue,21.9789,-159.3672,25.77,77,0,6.17,US,1694959812
18,18,cabo san lucas,22.8909,-109.9124,28.20,74,0,2.74,MX,1694959909
22,22,keelung,25.1283,121.7419,28.92,85,0,1.83,TW,1694959910
34,34,bethel,41.3712,-73.4140,20.42,68,0,0.00,US,1694959912
38,38,colonia,50.9333,6.9500,28.31,52,0,3.09,DE,1694959824


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
%%capture --no-display
# capture added to remove false error when creating empty column

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,lihue,US,21.9789,-159.3672,77,
18,cabo san lucas,MX,22.8909,-109.9124,74,
22,keelung,TW,25.1283,121.7419,85,
34,bethel,US,41.3712,-73.4140,68,
38,colonia,DE,50.9333,6.9500,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Remove rows with "No hotel found"
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found",:]

# Display sample data
hotel_df

Starting hotel search
lihue - nearest hotel: Kauai Palms
cabo san lucas - nearest hotel: Comfort Rooms
keelung - nearest hotel: 華國商務飯店
bethel - nearest hotel: Hampton Inn Danbury
colonia - nearest hotel: Wasserturm Hotel Cologne
mahanoro - nearest hotel: Vanilla Bungalow
talara - nearest hotel: Hospedaje El Aypate
alice springs - nearest hotel: Aurora Alice Springs
tolanaro - nearest hotel: Hôtel Mahavokey
arraial do cabo - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
tsiombe - nearest hotel: No hotel found
kalecik - nearest hotel: No hotel found
magong - nearest hotel: 元泰大飯店
nantucket - nearest hotel: No hotel found
ivankiv - nearest hotel: Надія
tinogasta - nearest hotel: No hotel found
ellsworth - nearest hotel: The Sanctuary B&B
siret - nearest hotel: No hotel found
kapa'a - nearest hotel: Pono Kai Resort
prieska - nearest hotel: No hotel found
southwick - nearest hotel: No hotel found
campo mourao - nearest hotel: Hotel Loydi


,City,Country,Lat,Lng,Humidity,Hotel Name
16,lihue,US,21.9789,-159.3672,77,Kauai Palms
18,cabo san lucas,MX,22.8909,-109.9124,74,Comfort Rooms
22,keelung,TW,25.1283,121.7419,85,華國商務飯店
34,bethel,US,41.3712,-73.4140,68,Hampton Inn Danbury
38,colonia,DE,50.9333,6.9500,52,Wasserturm Hotel Cologne
54,mahanoro,MG,-19.9000,48.8000,74,Vanilla Bungalow
70,talara,PE,-4.5772,-81.2719,69,Hospedaje El Aypate
77,alice springs,AU,-23.7000,133.8833,18,Aurora Alice Springs
92,tolanaro,MG,-25.0319,46.9987,67,Hôtel Mahavokey
104,wailua homesteads,US,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)